# Create decision variables quickly.

In this notebook, we will see how to create decision variables quickly in a python solver (iterer).

## Summary

1. <b> Basic method </b>
2. <b> Method with tables </b>
3. <b> Method with the python pulp dictionary </b>
4. <b> Create constraints faster with pulp. </b> dictionaries
5. <b> A tip to recover the names of the decision variables. </b>
6. <b> Create decision variables by combining two tables. </b>
    - method of use
7. <b> iterer </b>
    - With the Python Gekko solver
    - With the Python Cplex solver
    - With the Python CVXOPT solver

Global study proposed by <b> Estelle Derrien - GitHub Estelle15000 </b>

# 1. Basic method

Reminder :
A decision variable contains a value which will then be calculated by the solver to be the best possible value to minimize or maximize an objective function (this is optimization)

In [1]:
# La méthode de base :

# Import du solveur
import pulp as p

# On sette le problème
Lp_prob = p.LpProblem('Problem', p.LpMaximize)

# -----------------------------------
# On définit nos variables de décision ci-dessous
# -----------------------------------

x = p.LpVariable("x", lowBound=0, cat='Integer')   # Créer une variable x >= 0
y = p.LpVariable("y", lowBound=0, cat='Integer')   # Créer une variable y >= 0

# On les affiche
print(x)
print(y)

Restricted license - for non-production use only - expires 2024-10-28
No parameters matching '_test' found
x
y


# 2. Method with tables

In the previous example, it goes because we have only two variables of decisions, but what will be if we have 100?

In [2]:
# La méthode rapide :

import pulp as p

Lp_prob = p.LpProblem('Problem', p.LpMaximize)

# On crée d'abord le nom de nos variables de décision

objets = [
    'objet_1',
    'objet_2',
    'objet_3',
    'objet_4',
    'objet_5',
]

# On crée les variables, celles ci seront binaires. En fait, on itère sur le tableau objets et i est l'index.
x = {i: p.LpVariable(name=f"{i}", lowBound=0, cat='Binary') for i in objets}

# Maintenant, on peut acceder aux variables de cette façon : x['objet_1']

# On peut ensuite créer un deuxième lot de variables de décision, qui cette fois ci, seront entières.

usines = [
    'usine_1',
    'usine_2',
    'usine_3',
    'usine_4',
]

# On crée les variables, celles ci seront entières. En fait, on itère sur le tableau objets et k est l'index.
m = {k: p.LpVariable(name=f"{k}", lowBound=0, cat='Integer') for k in usines}

# On accède ensuite à une variable comme ça, ou avec des boucle for
print(m['usine_1'])


usine_1


# 3. Method with the python pulp dictionary

In [3]:
# Import the PuLP lib
from pulp import *

# Créer le type de problème
prob = LpProblem ("MaximiserProfit", LpMaximize)

# La liste de nos produits
produits = ["automobile", "cycle1","cycle2","dragon","nounours","poupee","arc"]

# Ici, on crée nos variable de décision en créant un "dicts".
x = LpVariable.dicts("produits ", produits , 0)

# On visualise notre dicts
print(x)


{'automobile': produits__automobile, 'cycle1': produits__cycle1, 'cycle2': produits__cycle2, 'dragon': produits__dragon, 'nounours': produits__nounours, 'poupee': produits__poupee, 'arc': produits__arc}


# 4. Create constraints faster with dictionaries

In this example below, constraints are created using Python dictionaries in order to <b> iterate </b>, to go faster.

In fact, it is also an association of values with our decisions variables, made in Python Dictionary format, which allows you to write faster then in the objective function and in the constraints.

But beware, the number of decisions variables must correspond to the number in Python dictionaries, here, we have 7 decisions variables, therefore in dictionaries, we find this size of 7 decision variables.

This can make things easier in the event that this data is important from a database, it becomes easier to place it in a dictionary.


In [4]:
# Import the PuLP lib
from pulp import *

# Créer le type de problème, ici, on veut maximiser notre profit de notre usine de jouets
prob = LpProblem ("MaximiserProfit", LpMaximize)

# La liste de nos produits ( des jouets), ca va être les variables de décision, 
# elles pourront prendre une valeur entière, vu que ce sont des objets uniques. 
# Par exemple , produire 15 automobiles, produire 25 arcs etc ...
produits = ["automobile", "cycle1","cycle2","dragon","nounours","poupee","arc"]

# Les bénéfices en EUROS par produits
benefices = {"automobile": 8, "cycle1": 12, "cycle2": 14,"dragon": 3,"nounours":6,"poupee":13,"arc":12}

# Emplois (en kgs)
plastique = {"automobile": 2, "cycle1": 4, "cycle2": 5,"dragon": 3,"nounours":1,"poupee":4,"arc":2}
bois      = {"automobile": 1, "cycle1": 1, "cycle2": 2,"dragon": 2,"nounours":1,"poupee":5,"arc":1}
acier     = {"automobile": 1, "cycle1": 2, "cycle2": 3,"dragon": 3,"nounours":2,"poupee":2,"arc":5}


# Les noms de nos ressources
ressources = {"plastique", "bois", "acier"}

# Les stocks de nos ressources en KG
stocks = {"plastique": 142, "bois ": 117, "acier": 124}

# On crée nos variables, en se basant sur le tableau x ( array en Anglais)
# On stipule que ce sont des variables entières, normal, puisque ce sont des jouets (On ne peut pas avoir 1/2 jouet...).
x = LpVariable.dicts("produits ", produits , lowBound=0, cat='Integer')

# La fonction objectif, Maximiser le bénéfice.
# Ici, on voit qu'on itère à l'aide de notre tableau produits, sur les bénéfices qui sont contenus dans un 
# dictionnaire Python. Cette méthode revient souvent sur StackOverflow et en général.
prob += lpSum([benefices[i] * x[i] for i in produits ]), "MaximiserBenefice" 

# Nos contraintes.
# On respecte notre production sous contrainte de stocks
# Ici, n voit qu'on itère à l'aide de notre tableau produits, sur les stocks qui sont contenus dans un 
# dictionnaire Python. Cette méthode revient souvent sur StackOverflow et en général.
prob += lpSum([plastique[i] * x[i] for i in  produits]) <= 142 ,"MaxPlastique"

# La ligne suivante veut dire : 
# "" Prends chaque valeur du tableau bois indicé par i et multiplie le par la 
# variable de décision x indice i, fait la somme de tout
# et cette somme doit être inférieure à notre stock de bois de 117
prob += lpSum([bois[i]      * x[i] for i in  produits]) <= 117 ,"MaxBois"

prob += lpSum([acier[i]     * x[i] for i in  produits]) <= 124 ,"MaxAcier"

# Production minimale par produits pour les clients : 2 unités
for p in produits:
   prob += x[p] >= 2, f"min production units for product {p}"

# On écrit aussi le probleme dans un fichier
# prob.writeLP ( "JouetsModel.lp")

# On utilise le solver pulp
prob.solve()

# On affiche le statut de la solution
print ("Status:", LpStatus [prob.status])

# Afficher l'optimium de chaques variables produits qui s'exprime en unité construites
for v in prob.variables ():
    print (v.name, "=", v.varValue)


# Le résultat de la fonctioj objectif est ici :
print ("TotalProfit", value (prob.objective))
  

Status: Optimal
produits__arc = 2.0
produits__automobile = 40.0
produits__cycle1 = 2.0
produits__cycle2 = 2.0
produits__dragon = 2.0
produits__nounours = 26.0
produits__poupee = 2.0
TotalProfit 584.0


# 5. A tip to recover the names of the decision variables.

In this example, below, we see that the author directly writes the characteristics of decision variables in Python dictionaries, then he recovers the names of the decision variables using the order
items = list (sorted (v.keys ())).

Suddenly, it goes even faster to create the variables then in the command:
x = lpvariable.dicts ('x', items, lowbound = 0, upbound = 1, cat = lpinteger)

In [5]:
# knapsack-pulp.py

from pulp import *

v = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
limit = 14
items = list(sorted(v.keys()))

# Create model
m = LpProblem("Knapsack", LpMaximize)

# Variables
x = LpVariable.dicts('x', items, lowBound=0, upBound=1, cat=LpInteger)

# 6. Create decision variables by combining two tables.


In this example, we see that the author of Python Pulp creates decision variables by associating 2 tables: suppliers and customers.

It allows you to create all possible solutions using the for loop.

It is used, after optimization

In [6]:
import pulp as p

# Variables de décision
fournisseurs = ['A','B']
clients = ['1','2','3','4','5']
costs = [  
# clients   # 1  2  3  4  5
                [3, 1, 3, 2,9],      # A   fournisseurs
                [25, 15, 32, 22,8],  # B
]

# C'est un problème de minimisation
Problem = p.LpProblem('optimisation_transport',p.LpMinimize)

# Créer une liste de toutes les routes possibles
Routes = [(w, b) for w in fournisseurs  for b in clients]

# Créer un dictionnaire de variables de décisions des routes
vars = p.LpVariable.dicts("Route", (fournisseurs , clients), 0, None, p.LpBinary)

# Créer un dictionnaire des coûts de chaque routes
cost = p.makeDict([fournisseurs,clients], costs, 0)

# Fonction objectif
Problem += p.lpSum([cost[w][b]  * vars[w][b]  for (w, b) in Routes])

# Contrainte
Problem += (p.lpSum([ vars[w][b]  for (w, b) in Routes]) >= 5 , "minimum_de_5_routes")

# On résouds
Problem.solve()
 
# On imprime le résultat
print('Statut:', p.LpStatus[Problem.status])
print('Cout total minimisé = ', p.value(Problem.objective))
 
for i in Problem.variables():
    if i.varValue > 0:
        print('Choix de la route :',i.name, '=', i.varValue)


Statut: Optimal
Cout total minimisé =  17.0
Choix de la route : Route_A_1 = 1.0
Choix de la route : Route_A_2 = 1.0
Choix de la route : Route_A_3 = 1.0
Choix de la route : Route_A_4 = 1.0
Choix de la route : Route_B_5 = 1.0


The syntax of linear problems can be organized differently depending on the authors of the programs, here is the example of the aggregated production of Aaon Stubberfield where it also associates tables to create decision variables, but in a different way than the first example.

Especially since a code is always the application of a written mathematical model, but there, it becomes difficult, you have to get used to and take the time to read and understand.We are supposed to know how to go back and old between the mathematical model and the code, and vice versa.


In [7]:
from pulp import *
demand = {'A':[5,0,0],'B':[8,7,6]}
costs = {'A':[20,17,18],'B':[15,16,15]}

# On crée le problème
model = LpProblem("Aggregate_Production_Planning",LpMinimize)

# On définit les variables
time = [0, 1, 2]
prod = ['A','B']

# On crée un dictionnaire de toutes les variables de décision de productions possibles et leur temps
#### C'est là que l'on crée des variables de décision en associant les 2 tableaux.***
X = LpVariable.dicts("prod", [(p, t) for p in prod for t in time],lowBound=0, cat="Integer")
# On regarde l'association créee :
print(X)

# On crée la fonction objectif : minimiser les coûts
model += lpSum([costs[p][t] * X[(p, t)] for p in prod for t in time])

# On définit la contrainte que la production >= demande
for p in prod:
    for t in time:
        model += X[(p, t)] >= demand[p][t] 

# Résoudre
model.solve()

# On imprime les variables qui ont leur valeur optimisées
for v in model.variables():
    print(v.name, "=", v.varValue)
    
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Coût total = ", value(model.objective))

{('A', 0): prod_('A',_0), ('A', 1): prod_('A',_1), ('A', 2): prod_('A',_2), ('B', 0): prod_('B',_0), ('B', 1): prod_('B',_1), ('B', 2): prod_('B',_2)}
prod_('A',_0) = 5.0
prod_('A',_1) = 0.0
prod_('A',_2) = 0.0
prod_('B',_0) = 8.0
prod_('B',_1) = 7.0
prod_('B',_2) = 6.0
Coût total =  422.0


# 7. Iterate with Python Gekko, the non -linear solver.

It's different from with Pulp.

Link: https://stackoverflow.com/questions/64542594/how-could-constraints-be-my

Here is an example of cost reduction with iteration in working time constraint:

In [8]:
from gekko import GEKKO

# stored as list
my_vars = ['x1','x2']
# stored as dictionaries
Cost = {'x1':100,'x2':125}
Min = {'x1':0,'x2':0}
Max = {'x1':70,'x2':40}
Work = {'x1':50,'x2':50}

LP = GEKKO(remote=False)


va = LP.Array(LP.Var, (len(my_vars)))  # array

# Le truc qui diffère , en fait il crée un dictionnaire à l'aide du tableau créé juste avant :
vd = {}                                # dictionary
for i,xi in enumerate(my_vars):
    vd[xi] = va[i]
    vd[xi].lower = Min[xi]
    vd[xi].upper = Max[xi]


# Fonction coût
LP.Minimize(LP.sum([Cost[xi]*vd[xi] for xi in my_vars])) 


# On voit que l'itération fonctionne !
LP.Equation(LP.sum([Work[xi]*vd[xi] for xi in my_vars])>=200)


LP.solve(disp=False)

# On affiche le résultat
for xi in my_vars:
    print(xi,vd[xi].value[0])
print ('Cost: ' + str(LP.options.OBJFCNVAL))

x1 3.9999999998
x2 1.8225924919e-09
Cost: 400.00000021


# 7b.Iter with the Python Cplex solver